In [0]:
import pandas as pd
import numpy as np
import string

In [0]:
filename="dataset_compiled1506"
df=pd.read_csv(filename+".csv")
df.shape

In [0]:
df = df[df['numbermap'].notna()]
df

In [0]:
exp=list(df["expression"])
sig=[eval(k) for k in list(df["sigDict"])]
numbermap=[eval(k) for k in list(df["numbermap"])]
answer=[round(float(k),2) for k in list(df["answer"])]

In [0]:
def parentheses_match(input_string):
  s = []
  balanced = True
  index = 0
  while index < len(input_string) and balanced:
    token = input_string[index]
    if token == "(":
      s.append(token)
    elif token == ")":
      if len(s) == 0:
        balanced = False
      else:
        s.pop()
    index += 1
  return balanced and len(s) == 0

In [0]:
#Check Exp from Sig
level1=[]
print("Level 1")
print("=================================================")
for p,i in enumerate(sig):
  flag=True
  for key in i.keys():
    if i[key]:
      if exp[p].find(key)==-1:
        flag=False
        break
    else:
      if exp[p].find(key)!=-1:
        break
  level1.append(flag)
print("=================================================\n\n")

df["Level 1"]=level1


In [0]:
#Check Sig From Exp
level2=[]
print("Level 2")
print("=================================================")
for i in range(len(exp)):
  flag=True
  texp=exp[i]
  tsig=sig[i]
  truesig=[ x for x in tsig if tsig[x]]
  for ch in texp:
    if ch in string.ascii_uppercase:
      if not ch in truesig:
        flag=False
        print(texp,tsig)
        break  
  level2.append(flag)
print("=================================================\n\n")

df["Level 2"]=level2

In [0]:
#Check Brackets
level3=[]
print("Level 3")
print("=================================================")
for i in range(len(exp)):
  flag=parentheses_match(exp[i])
  if not flag:
      print(exp[i],sig[i])
  level3.append(flag)
print("=================================================\n\n")

df["Level 3"]=level3

In [0]:
#Check Answer
level4=[]
print("Level 4")
print("=================================================")
for i in range(len(numbermap)):
  flag=False
  print ("\n\nTry: ",i)
  try:
    if not (level1[i] and level2[i] and level3[i]):
      raise ValueError("Previous Test Was Failed.")
    nummap=numbermap[i]
    nummap.update({"^":"**"})
    cexp=exp[i]
    ans=answer[i]
    for key in nummap:
      cexp = cexp.replace(key,nummap[key])
    assert eval(cexp)==ans
    flag=True
    print("Valid: "+str(eval(cexp))+"  "+str(ans))
  except ValueError as e:
    print(e)
    flag=False
  except:
    print("Invalid: "+cexp+"  "+str(ans))
    flag=False
  finally:
    level4.append(flag)
print("=================================================\n\n")
df["Level 4"]=level4


In [0]:
df.to_csv(filename+"_qc.csv")